<img src="https://beyondthestates.com/wp-content/uploads/2023/09/download.png" width=20% height=20%>

# Higher Diploma in Data Analytics course from [ATU](https://www.atu.ie/) in Winter 2024/25
##  Programming for Data Analytics - project 2024/2025
***



<div style="text-align:center">
<img src="img/Wind_turbines.jpg" width=60% height=80%>
<div style="text-align:center; font-size:10px;"><b>by stockertui @vecteezy.com </b>
</div>

### Project Overview 
***

In this project I need to demonstrate a data analysis of the wind speed in one of the Ireland's regions.
I will analyze 70 years of historical wind speed data from the Roches Point weather station in County Cork, Ireland. The data, sourced from the [Met Éireann historical dataset](https://www.met.ie/climate/available-data/historical-data), spans from January 1955 to December 2024, providing a comprehensive range for long-term trend analysis.

This extensive time range is offering identification of long-term trends, seasonal variations, and other patterns relevant to wind speed and energy production.

Roches Point Station is located at the southeastern tip of County Cork, near the entrance to Cork Harbour, a strategic coastal location that often experiences significant wind activity.
The station records meteorological data such as wind speed, temperature, and precipitation at hourly intervals, making it suitable for detailed analysis.
Its coastal location provides valuable insights into wind patterns influenced by the Atlantic Ocean, making it relevant for assessing wind energy potential in Ireland.

Here's what the dataset contains: 

- date:  -  Date and Time (utc)
- rain:  -  Precipitation Amount (mm)
- temp:  -  Air Temperature (C)
- wetb:  -  Wet Bulb Temperature (C)
- dewpt: -  Dew Point Temperature (C)
- vappr: -  Vapour Pressure (hPa)
- rhum:  -  Relative Humidity (%) 
- msl:   -  Mean Sea Level Pressure (hPa)
- wdsp:  -  Mean Wind Speed (kt)
- wddir: -  Predominant Wind Direction (deg)
- ind:   -  Indicator

To perform the analysis I will need two variables `wdsp` and `wddir`. 


### Project Goal
***

The primary goal of this project is to analyze wind speed data `wdsp` to assess its potential for wind energy generation and explore historical trends and patterns. By focusing on one location and utilizing this long-term dataset, I will address the following key questions:

- How do wind speeds vary seasonally and annually at Roches Point?
- Are there noticeable trends in wind speeds over the past 70 years?
- What is the potential wind energy output for this location based on historical data?
- Are there seasonal or extreme wind events that impact energy reliability?


To perform this analysis I will use [Python](https://en.wikipedia.org/wiki/Python_(programming_language)) and its libraries: 

[Pandas](https://pandas.pydata.org/):
- Used to handle and preprocess the dataset (e.g., reading the CSV file, converting dates, grouping data).

[Matplotlib](https://matplotlib.org/):

- Used for creating plots to visualize trends and regression results.

[Scikit-Learn](https://scikit-learn.org/stable/index.html):

- Used to build and fit the linear regression model.

[NumPy](https://numpy.org/):

- Helps with numerical calculations.

### Importing the required Libraries
***


Import the pandas library. Pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool, built on top of the Python programming language.

In [16]:
import pandas as pd

NumPy is the next library I need. It is a fundamental package for scientific computing in Python that provides a multidimensional array object, various derived objects (such as masked arrays and matrices). 

In [17]:
import numpy as np

In [18]:
import matplotlib.pyplot as plt

In [19]:
import seaborn as sns

### Loading, Cleaning and Inspecting the Data
***
The dataset will be loaded into a Python environment using the `pandas` library. After loading, the data will be inspected for its structure, including column names, data types, and missing values etc.

Our dataset is saved in .csv file in this repository. After inspecting this data by reviewing the file, I need to ignore first 17 rows as they contain description elements and they are not part of the dataset, therefore they can be ignored. 

In [33]:
df = pd.read_csv('data/hly1075.csv', skiprows=17, low_memory=False) # https://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options
df.head(3)

,date,ind,rain,ind.1,temp,ind.2,wetb,dewpt,vappr,rhum,msl,ind.3,wdsp,ind.4,wddir
0,01-dec-1955 01:00,0,0.0,0,10.7,0,10.0,9.4,11.8,91,1002.1,1,16,1,170
1,01-dec-1955 02:00,0,2.9,0,9.8,0,9.7,10.0,12.0,99,1001.8,1,11,1,190
2,01-dec-1955 03:00,0,3.8,0,9.7,0,9.5,9.4,11.7,97,1001.7,1,9,1,160


Let's check the the data type of each column

In [26]:
df.dtypes
print(df.dtypes)

date     object
ind       int64
rain     object
ind.1     int64
temp     object
ind.2     int64
wetb     object
dewpt    object
vappr    object
rhum     object
msl      object
ind.3     int64
wdsp     object
ind.4     int64
wddir    object
dtype: object


As we can see both columns `wdsp` and `wddir` are showing as objects which means they might have contains NaN values. Let's check: 

In [30]:
# Check for missing values
nan_values = df.isna()
print(nan_values)

         date    ind   rain  ind.1   temp  ind.2   wetb  dewpt  vappr   rhum  \
0       False  False  False  False  False  False  False  False  False  False   
1       False  False  False  False  False  False  False  False  False  False   
2       False  False  False  False  False  False  False  False  False  False   
3       False  False  False  False  False  False  False  False  False  False   
4       False  False  False  False  False  False  False  False  False  False   
...       ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
585546  False  False  False  False  False  False  False  False  False  False   
585547  False  False  False  False  False  False  False  False  False  False   
585548  False  False  False  False  False  False  False  False  False  False   
585549  False  False  False  False  False  False  False  False  False  False   
585550  False  False  False  False  False  False  False  False  False  False   

          msl  ind.3   wdsp  ind.4  wdd

In [31]:
# Count missing values
df.isnull().sum()
print(df.isnull().sum())

date     0
ind      0
rain     0
ind.1    0
temp     0
ind.2    0
wetb     0
dewpt    0
vappr    0
rhum     0
msl      0
ind.3    0
wdsp     0
ind.4    0
wddir    0
dtype: int64


After using basic functions `df.isnull()` that shows True/False for missing values and `df.isnull().sum()` that shows total missing values per column, we can see that there is no missing/empty cells in this dataset. That would mean the cells that contain no data are not empty or zeroed, but they might contain a string placeholder for no-data cells. For more information you can look [here.](https://saturncloud.io/blog/how-to-check-if-a-particular-cell-in-pandas-dataframe-is-null/#:~:text=Checking%20for%20Null%20Values%20in%20Pandas%20DataFrame&text=To%20check%20for%20null%20values%20in%20a%20pandas%20DataFrame%2C%20we,cell%20is%20null%20or%20not.)

To find out, I will use the `unique()` function in Pandas. It is used to find all the distinct or unique values in a specific column. After running it, we can quickly understand the variety of values present in data, which in this case is a white space. For more information, you can look at documentation [here](https://www.w3resource.com/numpy/manipulation/unique.php#:~:text=unique()%20function%20is%20used,that%20reconstruct%20the%20input%20array)

In [34]:
# Get unique values in the Mean Wind Speed column
unique_wdsp = df['wdsp'].unique()

print(unique_wdsp)

['16' '11' '9' '5' '12' '15' '13' '14' '7' '4' '3' '6' '1' '0' '21' '19'
 '23' '22' '20' '8' '2' '17' '18' '10' '25' '29' '26' '24' '27' '28' '30'
 '31' '36' '32' '33' '34' '40' '41' '42' '38' '35' '46' '48' '44' '37'
 '39' '43' '47' '50' '53' '54' '45' '51' '55' '49' '52' '59' '56' ' ' '62'
 '60']


In [ ]:
# Get unique values in the Predominant Wind Direction column
unique_wddir = df['wddir'].unique()

print(unique_wddir)

['170' '190' '160' '140' '330' '340' '350' '320' '360' '10' '0' '230'
 '210' '200' '220' '250' '260' '280' '300' '240' '270' '310' '150' '180'
 '290' '40' '20' '50' '70' '60' '80' '110' '130' '100' '90' '120' '30' ' ']


As we can see in the range of unique values, in both columns appears to be `' '` value, which is a white space. That is why our data type was shown as object. To be able to use this data for analysing, I need to clean this data. 
I will create a new dataset `cleandf`. It will replace white spaces with `NaN` and then drop them from my dataset. 



In [40]:
cleandf = df[['wddir', 'wdsp']].copy()  # Make a copy of the dataframe

# Replace spaces with NaN for both columns
cleandf['wdsp'] = cleandf['wdsp'].replace(' ', np.nan)
cleandf['wddir'] = cleandf['wddir'].replace(' ', np.nan)

# Drop rows with NaN in either 'wdsp' or 'wddir'
cleandf.dropna(subset=['wdsp', 'wddir'], inplace=True)

# Convert 'wdsp' and 'wddir' to float
cleandf['wdsp'] = cleandf['wdsp'].astype(float)
cleandf['wddir'] = cleandf['wddir'].astype(float)

In [41]:
# Check for missing values
nan_values = cleandf.isna()
print(nan_values)

        wddir   wdsp
0       False  False
1       False  False
2       False  False
3       False  False
4       False  False
...       ...    ...
585546  False  False
585547  False  False
585548  False  False
585549  False  False
585550  False  False

[584955 rows x 2 columns]


In [42]:
# Count missing values
cleandf.isnull().sum()
print(cleandf.isnull().sum())

wddir    0
wdsp     0
dtype: int64


In [43]:
# Checking the data type of each column
cleandf.dtypes
print(cleandf.dtypes)

wddir    float64
wdsp     float64
dtype: object
